# A Deeper Understanding: Stochastic Graident Descent and Backpropagation

You’ve built some very powerful models and I hope you’re excited by the new things you can do. At this point, I’d like to loop back to some technical details that will help you keep growing and doing new things. 

In this lesson, you’ll learn how we use stochastic gradient descent to set the numeric values in both dense and convolutional layers, as well as how arguments like batch size and number of epochs affect this model training process.

We’ll start with an example using Dense layers. To simplify the explanation, we’ll take a brief detour to working with tabular data, the type of data you might manipulate with Pandas or a SQL database.

Imagine we want to assess someone’s risk of developing diabetes in the next year. We have data about their age, weight, and a blood sugar measurement. We could draw a layer with one node for each variable.

<img src="https://i.imgur.com/6AxgtUI.png">

Since this layer represents raw data, we call this the input layer. For any given individual, we can fill in their values, which are just numbers.  Let’s connect these directly the prediction layer.  Each person either develops diabetes in the next year, or they don’t. So there are two possible outcomes, and thus two nodes in the prediction or output layer.  We’ll connect these as a dense layer like you’ve already seen, where each input is connected to each output.  But to be concrete, each connection will have a number associated with it. The numbers are called weights. I’ll fill in some example weights here.

<img src="https://i.imgur.com/d4mKWM8.png">



For concreteness, let’s consider a single person. Our example person is 50 years old, weighs 80 kg, and has a blood sugar measurement of 80mg/dL. We can calculate the values in the next layer using a process called forward propagation. This is conceptually similar to the calculations you did with convolutions in the first video.

The top node of the output layer has 3 connections coming into it. For each one, we multiply the weight by the value at the node the connection comes from.  For example, at the top node we have 0.1 times 50 from the age node, plus 0.1 times 80 from the weight node, minus 0.1 * 80 from blood sugar.  We sum those to get 5. We do the same thing at the bottom node, and that would give us 0 from the age and weight nodes, because those have a weight of zero. But we get 0.1 times 80 from blood sugar, for a total of 8.

<img src="https://i.imgur.com/4dwAl1r.png">

Now we have a score at both the Yes and No nodes of output. The softmax function converts those into probabilities. TensorFlow handles the calculations, so you don’t need to remember the softmax function. In case you are curious, putting those through the softmax function gives a value of 0.05 or 5 percent for the top node, and 95% for the bottom node.  So the network predicted this person is 5 percent likely to get diabetes.

When working with tabular data, it’s common to have many dense layers between the input and the output, the same way we had many layers of convolutions when working with images. The layers between the input and the output are called hidden layers.  The forward propagation process remains the same as what you’ve already seen.  In this case I calculate this hidden layers values from the input layer.  If there were multiple hidden layers, I would keep filling in values from left to right. Eventually I get to the output layer, and I do the same type of arithmatic calculation.  

<img src="https://i.imgur.com/ii80muN.png">

This is a slight simplification of what’s normally done, because in practice we’ll apply some non-linear function at each node in the hidden layers. Including a non-linear function helps the model capture both non-linearities and interactions impacts between variables better.  The most common function to apply is the ReLU or rectified linear function. I have a link with detail about that below, but the topic is a tangent from what we’re focused on at the moment.



We use forward propagation to make predictions, but we want to understand where the weights on the connections come from.  After all, changing the weights will change your predictions. Good weights are the key to accurate predictions.

There are 3 key concepts to understanding how the model gets good weights. Those are loss functions, gradient descent and back propagation. The loss function and gradient descent showed up in our code as arguments to the compile function we ran before fitting our model.

## The Loss Function

The loss function measures how good a model’s predictions are. The loss function’s arguments are the actual values of the target we wanted to predict, as well as the model’s predictions.  As a convention, we use loss functions where lower scores are better.  So, if the predicted values are close to the the actual values of the target, the loss function will give a low value.  If the predictions are usually far off, the loss function will return a high value.

For any set of data, whether the model is accurate or inaccurate will depend on the model weights. So, the numeric value of the loss function on any dataset will also change if we change the model’s weights.

This fact is the key to our model optimization procedure: gradient descent and stochastic gradient descent. We’ll use stochastic gradient descent to set the weights that minimize our loss function.

## Stochastic Gradient Descent

Many people describe gradient descent with an analogy. Again, remember that the weights affect our predictions, and thus they affect the loss function. So if we had a model with only two weights, we could plotting the loss function on a graph against two weights.  In practice, we have more than two weights in most models, but I couldn’t draw a graphic with more dimensions than this.  So we’ll stick with two weights and a loss function for the sake of this explanation.

<img src="https://i.imgur.com/Qm8UlP3.png?1">

In this plot, we have the loss function on the vertical or z axis, and we have weights on the other two axes. I want to find the low point on surface, because that’s the point with the lowest value of the loss function, and thus the most accurate model.  The process is a little bit like being stuck in a field with lots of hills and valleys, in complete dark, and trying to find the lowest point.  

How would you do it?  You might feel which direction goes downhill most steeply, and take a step in that direction.  Then you’d feel around again to find which direction is downhill, and you’d take another step.  You repeat this until you can’t go down any more.  This is basically how gradient descent works.  You look at data, see which way you can change the weights to get a lower loss function, and you change the weights slightly in that direction, to get better weights.  Then you repeat this to improve the weights slightly again.  How do you find which way you can change the weights to improve the loss function?  Basically, how do you see which way goes downhill?  That is backward propagation, which we’ll get to in a minute.  Before we do, a couple more details are worth pointing out in gradient descent.

First, we generally don’t use all of our data to calculate each step.  Doing so would require a lot of calculations, so it would be slow.  Instead, we look at some of the data at a time.  The “batch size” is the number of rows of tabular data we use to calculate each step. Or if we are using images for data, the batch size is the number of images used to calculate each step.  You set the batch size as an argument in your transfer learning program.

We take one small batch, and then the next, until we’ve used all our data.  One time through all the data is called an epoch.  We incrementally improving weights for multiple epochs, so each image or data point would be used to improve weights more than once.  We set the number of epochs as an argument to the fit command, and you saw that in the code we wrote when trying out data augmentation.

# Back-propagation

Back-propagation is the process by which we find out which way to change the weights at each step of gradient descent. Some people will be interested in learning the details.  Others will want only a very general intuition for it since TensorFlow does back-propagation for you. I’ll give a quick overview here. Here's [a link with more detail](https://colah.github.io/posts/2015-08-Backprop/) for those who want it.

Let’s go back to the network with a hidden layer. We used forward propagation to go from input data, to values in the hidden layer, and then to predictions.  For each person in our training data, we have a value for whether they actually got diabetes.  

Let’s consider how we would change the weights to make the model more accurate for a batch of 1, and let’s say the actual value of the prediction target for this person said they didn’t get diabetes. If we predicted they wouldn’t get diabetes, our loss function will take a low value, with little ability to improve further.  If we predicted they were likely to get diabetes, we would have a lot of room to improve.  But how do we change the weights to improve?

We’ll go back one layer at a time.  Consider a node feeding into that value, which had a positive value during forward propagation.  

<img src="https://i.imgur.com/D5wCoN7.png">

There is a weight connecting that node to the output for no.  The higher that weight is, the more likely we are to predict no. That means the model would have a better loss function for this person.  So, we can increase the value on this red weight, and that improves the loss function.  The more we increase it, the more we’ll improve for this person. But it also might cause us to be very wrong for other people in the dataset.  So, we’ll probably make only a small change in this weight, corresponding to a small step in that field where we were looking for the low point.

Incidentally, lowering the weight from this hidden node to the “yes” prediction has a similar effect, so we’d do that too.

<img src="https://i.imgur.com/V7Rskae.png">

Even aside from changes in that last set of weights, the value in the hidden node may have contributed to making our predictions either more or less accurate. That is, raising or lowering the value in that node might improve our predicted probabilities.  We can change the value in that node by changing the weights that feed into it. If we had many layers, we would continually iterate back from the right to the left, until we got to the leftmost set of weights.

<img src="https://i.imgur.com/uSiw03y.png">

The mathematics of calculating how to change weights that are far back involves the chain rule from calculus, and each time we move leftward in the diagram, TensorFlow uses a calculation it already did for changing weights on the right.

The size of weight changes is determined by something called the learning rate.  Low learning rates mean that your model may take a long time training before it gets accurate. High learning rates mean your model may take huge steps around that field, always jumping over the best weights and never getting very accurate.

If don’t specify an optimization method, or if you specify SGD as your optimizer, you use the default learning rate of 0.01.  This is a reasonable value.  But if your model isn’t improving as it trains, you could try a different value, from half that to about 0.03 or so can be reasonable.  

Even better, you can use the argument `optimizer="adam"` for a special variation of gradient descent that automatically figures out the best learning rate throughout the gradient descent process.

While we demonstrated this with dense layers, it works the same with convolutional layers.  Each weight in each layer has an impact on the prediction.  We iteratively make small updates to those weights trying to find the low point in our loss function. Otherwise, move on and we’ll create a full model from scratch.


# Keep Going
Now you are ready to **[train your own models from scratch](https://www.kaggle.com/dansbecker/deep-learning-from-scratch).**

---
**Links Mentioned**

[ReLU activation function](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning)